In [ ]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import astropy.constants as cte
import astropy.units as un
from astropy.cosmology import Planck15 as cosmo
import emcee
import scipy
from scipy import optimize as op
import corner
import os
os.environ["OMP_NUM_THREADS"] = "8"
from multiprocessing import Pool
import time
from datetime import datetime 

from scipy.optimize import minimize

In [ ]:
c=cte.c
c_um=c.to("um/s")
c_mm=c.to("mm/s")

c = c.value # m/s

Msun=cte.M_sun
Msun = Msun.value # kg

h = cte.h.value

K=cte.k_B.value

In [ ]:
file = open("/Users/mariajesusfloresmoraga/Desktop/DSFG/book2.csv")
csvreader = csv.reader(file)
header = []
header = next(csvreader)

rows = []
for row in csvreader:
        rows.append(row)

ID= []

z=[]

S3=[]
errS3=[]

S2=[]
errS2=[]

S1=[]
errS1=[]

S870=[]
errS870=[]

S500=[]
errS500=[]

S350=[]
errS350=[]

S250=[]
errS250=[]

S160=[]
errS160=[]

S100=[]
errS100=[]

for j in rows:   
    
    ID.append(j[0])
    z.append(float(j[1]))
    S3.append(float(j[2]))
    errS3.append(float(j[3]))
    S2.append(float(j[4]))
    errS2.append(float(j[5]))
    S1.append(float(j[6]))
    errS1.append(float(j[7]))
    S870.append(float(j[8]))
    errS870.append(float(j[9]))
    S500.append(float(j[10]))
    errS500.append(float(j[11]))
    S350.append(float(j[12]))
    errS350.append(float(j[13]))
    S250.append(float(j[14]))
    errS250.append(float(j[15]))
    S160.append(float(j[16]))
    errS160.append(float(j[17]))
    S100.append(float(j[18]))
    errS100.append(float(j[19]))

In [ ]:
v__=np.array([500,350,250,160,100])*un.um
v_=(c_um/v__).to("Hz")
v=v_.value
l__=np.array([95,150,220])*un.GHz
l_=l__.to("Hz")
l=l_.value
vf_8d=np.concatenate((l,v))
print(vf_8d)

In [ ]:
frecsT_l=[]
errfT_l=[]
for m in range (len(rows)):
    
    frec=[S3[m],S2[m],S1[m],S500[m],S350[m],S250[m],S160[m],S100[m]]
    errf=[errS3[m],errS2[m],errS1[m],errS500[m],errS350[m],errS250[m],errS160[m],errS100[m]]
    frecsT_l.append(frec)
    errfT_l.append(errf)
    
datos=[]  #todas estas listas son iguales. Definí varias, para después evitar borrar datos donde no queremos
datos_2=[]


for x,y in zip(frecsT_l,errfT_l):
    datos_=[]
    datos_.append(x)
    datos_.append(y)
    datos.append(datos_) #así, la lista datos está formada por muchos pares de listas. En cada par está flux y err_flux
    datos_2.append(datos_)
 

In [ ]:
vf_80_8d=[]  #lista con 80 arrays de vf (vf con 8 datos)
for x in range(len(rows)):
    vf_80_8d.append(vf_8d)
    
vf_80_6d=[]  #lista con 80 arrays de vf (vf con 6 datos)
for x in range(len(rows)):
    l1=np.delete(vf_80_8d[x],7)
    l2=np.delete(l1,6)
    vf_80_6d.append(l2)

In [ ]:
datos6=[]
datos_26=[]

el2=[]
el2_err=[]
for x in range(len(rows)):
    
    el1_=np.delete(frecsT_l[x],7)
    el2_=np.delete(el1_,6)
    el2.append(list(el2_))
    
    el1_err_=np.delete(errfT_l[x],7)
    el2_err_=np.delete(el1_err_,6)
    el2_err.append(list(el2_err_))
    



for x,y in zip(el2,el2_err):
    datos_6=[]
    datos_6.append(x)
    datos_6.append(y)
    datos6.append(datos_6) #así, la lista datos está formada por muchos pares de listas. En cada par está flux y err_flux
    datos_26.append(datos_6)

In [ ]:
for i in range(len(datos)):  #ejemplo de como queda la lista
    print("Para {}; flujos:{}, errores:{}".format(ID[i],datos[i][0],datos[i][1]))

In [ ]:
for i in range(len(datos)):  #ejemplo de como queda la lista
    print("Para {}; flujos:{}, errores:{}".format(ID[i],datos6[i][0],datos6[i][1]))

In [ ]:
f_arr = []
e_arr = []
vf_arr = []

for i in range(len(rows)):
    masknan = ~np.isnan(datos_2[i][0])
    f = np.asarray(datos_2[i][0])
    e = np.asarray(datos_2[i][1])
    vf = np.asarray(vf_80[i])
    
    f_arr.append(f[masknan])
    e_arr.append(e[masknan])
    vf_arr.append(vf[masknan])

In [ ]:
f_arr6 = []
e_arr6 = []
vf_arr6 = []

for i in range(len(rows)):
    masknan6 = ~np.isnan(datos_26[i][0])
    f6 = np.asarray(datos_26[i][0])
    e6 = np.asarray(datos_26[i][1])
    vf6 = np.asarray(vf_80_6d[i])
    
    f_arr6.append(f6[masknan6])
    e_arr6.append(e6[masknan6])
    vf_arr6.append(vf6[masknan6])

In [ ]:
def tau(z,M_,d_,vf): 
    vf=vf*un.Hz
    d_=d_*un.kpc
    v0=353*un.GHz
    b=2
    M=Msun * 10** M_
    k0=0.15*(un.m**2/un.kg) 
    tau=k0*(vf/v0)**b *(z+1)**b * M/(np.pi*(d_/2)**2)
    return tau.to("")

In [ ]:
def P(vf,T,z):
    vf=vf*un.Hz
    T=T*un.K
    h=cte.h
    K=cte.k_B
    A=2*h/(c**2)
    a=(h*vf*(1+z)/(K*T))
    B=vf**3/(np.e**(a.value) -1)
    P=(A*B).to("mJy")
    return P

In [ ]:
def A(z,Tc,M_,d_,vf): 


    v0=353*un.GHz
    b=2
    
    Tc=Tc*un.K
    Tvariable=(np.linspace(Tc.value,1000,10000)) *un.K
    
    integrales=[]
    
    for i in vf:
        ARG = modelSv(i,z,Tvariable,M_,d_)*un.mJy *(Tvariable)**-7
        integrales.append(scipy.integrate.trapz(ARG,x=Tvariable))
    
    return integrales

In [ ]:
zD = np.linspace(0.,1000,100000)
D = scipy.interpolate.interp1d(zD, cosmo.angular_diameter_distance(zD).to("m").value, kind='linear')

def modelSv(vf,z,T,M_,d_):
    
    vf=vf # Hz
    d_=d_*3.08567758128e19 #kpc to m
    v0=353.0*1e9 #GHz to Hz
    b=2.
    M=Msun * 10.** M_
    k0=0.15 # m**2 / kg
    tau=k0*(vf/v0)**b *(z+1.)**b * M/(np.pi*(d_/2)**2)

    A=2*h/(c**2)
    a=(h*vf*(1+z)/(K*T))
    B=vf**3/(np.e**(a) -1)
    P=A*B
    
    b=2
    D_ = D(z)

    TAU=tau
    PL=P

    O=np.pi*(d_/2)**2 * (D_**-2)
    
    S_=O*(1-np.e**-TAU)*PL*1e29

    return S_

In [ ]:
def ml_S(ID,Flux,errFlux): #Single
    
    
    nll = lambda *args: -lnlike(*args)
    
    bnds = ((1,10),(None,None),(8,10.5),(2,7),(-np.inf,-1))
    
    sol = minimize(nll, [4.,55.,9.5,3.5,-1], args=(vf, Flux, errFlux,"S"),bounds=bnds)
    z_fit, T_fit, M_fit, d_fit, lnf_fit = sol.x
    
    
    plt.figure(figsize=(12,9))

    pm = [z_fit,T_fit,M_fit,d_fit]
    
    x = np.linspace(90,2000,100)*un.GHz
    x = x.to("Hz")
    x = x.value

    y = modelSv(x,pm[0],pm[1],pm[2],pm[3])

    plt.plot(x,y,label="MaxL")
    plt.plot(vf,np.array(Flux),marker="o",label="Data")
    plt.xscale("log")
    plt.yscale("log")
    plt.annotate(r"$z:{},T:{},M:{},d:{}$".format(np.round(z_fit,2),np.round(T_fit,2),np.round(M_fit,2),np.round(d_fit,2)),(5*10**11,5),fontsize=14)

    plt.ylabel("Flux (mJy)",fontsize=14)
    plt.xlabel("Frequency (Hz)",fontsize=14)
    plt.title("MaxL: {}".format(ID),fontsize=14)
    plt.legend()
    plt.savefig("fit_S_{}".format(ID),dpi=300,format="png")
    return(z_fit, T_fit, M_fit, d_fit)

In [ ]:
def lnprior(theta,model):
    z,T,M,d,lnf = theta
    if T>2.3 and z>0 and M>0 and d>0 and lnf<=-0.01:
        pM= -1 * (M - 9.5)**2 / (2 *0.5**2) #prior M (sacado de Su)
        #pZ= -1 * (z - 3)**2 / (2 * 2**2) #prior Z 
        #pd= -1 * (d - 4)**2 / (2 * 2**2) #prior T
        if model=="S":
            pT= -1 * (T - 55)**2 / (2 * 10**2) #prior T
        if model=="PL nolim":
            pT= -1 * (T - 55)**2 / (2 * 10**2) #prior Tc (como Tc=T-10, automaticamente restamos 10)
        if model=="PL lim":
            pT= -1 * ((T-10) - 35)**2 / (2 * 20**2) #prior Tc
        #return pM+pZ+pd+pT
        return pM+pT
    return -np.inf

In [ ]:
def lnlike(theta,x,y,yerr,model):
    z,T,M_,d_,lnf = theta
    y=np.array(y)
    yerr=np.array(yerr)
    #Sv = modelSv(x,z,T,M_,d_)
    #Sobs = modelSobs(x,z,T,M_,d_)
    #x = x*un.Hz
    if model=="S": 
        Sv = modelSv(x,z,T,M_,d_)
        inv_sigma2 = 1.0 / (yerr**2 + (Sv)**2 * np.e**(2*lnf))
        lnlike= -0.5 * np.sum((y-(Sv))**2*inv_sigma2 - np.log(inv_sigma2))
    else: 
        Sobs = modelSobs(x,z,T,M_,d_)
        inv_sigma2 = 1.0 / (yerr**2 + (Sobs)**2 * np.e**(2*lnf))
        lnlike= -0.5 * np.sum((y-(Sobs))**2*inv_sigma2 - np.log(inv_sigma2))
        
    return lnlike

In [ ]:
def lnprob(theta, x, y, yerr,model):
    lp = lnprior(theta,model)
    if not(np.isfinite(lp)):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr,model)

In [ ]:
def mcmc_S(ID,i,frecsT,errfT,model,w): #recibe con unidades
    x = np.linspace(90,2000,100)*un.GHz
    x = x.to("Hz")
    x = x.value
    
    Flux = frecsT
    errFlux = errfT
    
    
    nll = lambda *args: -lnlike(*args)
    
    bnds = ((1,10),(None,None),(8,10.5),(2,7),(-np.inf,-1))
    
    sol = minimize(nll, [4.,55.,9.5,3.5,-1], args=(vf_arr[i], f_arr[i], e_arr[i],"S"),bounds=bnds)
    z_fit, T_fit, M_fit, d_fit,lnf_fit = sol.x
    theta = [z_fit,T_fit,M_fit,d_fit,lnf_fit]
    
    print(theta)
    
    plt.figure(figsize=(12,9))


        
    pm = [z_fit,T_fit,M_fit,d_fit]

    y  = modelSv(x,pm[0],pm[1],pm[2],pm[3])

    ndim, nwalkers = 5, w
    pos = [theta + 1e-4 * np.random.randn(ndim) for i in range(nwalkers)] 
    
    with Pool() as pool:
    
        sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob ,args=(vf_arr[i],f_arr[i],e_arr[i],model),
                                pool=pool)
    
        start = time.time()
        sampler.run_mcmc(pos, 6000)
        end = time.time()
        multi_time = end - start
        print("Multiprocessing took {0:.1f} minutes".format(multi_time/60))
    
    samples = sampler.chain[:, 500:, :].reshape((-1, ndim))
    
    z_mcmc=[]
    T_mcmc=[]
    M_mcmc=[]
    d_mcmc=[]
    
    for i in range(ndim): #4 es ndim
        q = np.diff(np.percentile(samples[:, i], [16, 50, 84]))
        if i==0:
            z_mcmc.append([np.percentile(samples[:, 0], [16, 50, 84])[1],q[0],q[1]])
        if i==1:
            T_mcmc.append([np.percentile(samples[:, 1], [16, 50, 84])[1],q[0],q[1]])
        if i==2:
            M_mcmc.append([np.percentile(samples[:, 2], [16, 50, 84])[1],q[0],q[1]])
        if i==3:
            d_mcmc.append([np.percentile(samples[:, 3], [16, 50, 84])[1],q[0],q[1]])
            
    print("z_mcmc:",z_mcmc) 
    print("T_mcmc:",T_mcmc)
    print("M_mcmc:",M_mcmc)
    print("d_mcmc:",d_mcmc)
    
    zd = np.percentile(samples[:, 0], [0.1,99.5])[0]
    zu = np.percentile(samples[:, 0], [0.1,99.5])[1]

    Td = np.percentile(samples[:, 1], [0.1,99.5])[0]
    Tu = np.percentile(samples[:, 1], [0.1,99.5])[1]

    Md = np.percentile(samples[:, 2], [0.1,99.5])[0]
    Mu = np.percentile(samples[:, 2], [0.1,99.5])[1]

    dd = np.percentile(samples[:, 3], [0.1,99.5])[0]
    du = np.percentile(samples[:, 3], [0.1,99.5])[1]

    fig = corner.corner(samples[:,0:4],
                    labels=["z", "T", r"$M$", r"$d$"],
                    quantiles=[0.16, 0.5, 0.84],range=[(zd,zu),(Td,Tu),(Md,Mu),(dd,du)],
                    show_titles=True,
                    truths=[z_fit,T_fit,M_fit,d_fit])
    
    plt.savefig("Model Single {}.png".format(ID), dpi=300, format='png')
   
    return (z_mcmc,T_mcmc,M_mcmc,d_mcmc)

In [ ]:
def mcmc_S6(ID,i,frecsT,errfT,model,w): #recibe con unidades
    x = np.linspace(90,2000,100)*un.GHz
    x = x.to("Hz")
    x = x.value
    
    Flux = frecsT
    errFlux = errfT
    
    
    nll = lambda *args: -lnlike(*args)
    
    bnds = ((1,10),(None,None),(8,10.5),(2,7),(-np.inf,-1))
    
    sol = minimize(nll, [4.,55.,9.5,3.5,-1], args=(vf_arr6[i], f_arr6[i], e_arr6[i],"S"),bounds=bnds)
    z_fit, T_fit, M_fit, d_fit,lnf_fit = sol.x
    theta = [z_fit,T_fit,M_fit,d_fit,lnf_fit]
    
    print(theta)
    
    plt.figure(figsize=(12,9))


        
    pm = [z_fit,T_fit,M_fit,d_fit]

    y  = modelSv(x,pm[0],pm[1],pm[2],pm[3])

    ndim, nwalkers = 5, w
    pos = [theta + 1e-4 * np.random.randn(ndim) for i in range(nwalkers)] 
    
    with Pool() as pool:
    
        sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob ,args=(vf_arr6[i],f_arr6[i],e_arr6[i],model),
                                pool=pool)
    
        start = time.time()
        sampler.run_mcmc(pos, 6000)
        end = time.time()
        multi_time = end - start
        print("Multiprocessing took {0:.1f} minutes".format(multi_time/60))
    
    samples = sampler.chain[:, 500:, :].reshape((-1, ndim))
    
    z_mcmc=[]
    T_mcmc=[]
    M_mcmc=[]
    d_mcmc=[]
    
    for i in range(ndim): #4 es ndim
        q = np.diff(np.percentile(samples[:, i], [16, 50, 84]))
        if i==0:
            z_mcmc.append([np.percentile(samples[:, 0], [16, 50, 84])[1],q[0],q[1]])
        if i==1:
            T_mcmc.append([np.percentile(samples[:, 1], [16, 50, 84])[1],q[0],q[1]])
        if i==2:
            M_mcmc.append([np.percentile(samples[:, 2], [16, 50, 84])[1],q[0],q[1]])
        if i==3:
            d_mcmc.append([np.percentile(samples[:, 3], [16, 50, 84])[1],q[0],q[1]])
            
    print("z_mcmc:",z_mcmc) 
    print("T_mcmc:",T_mcmc)
    print("M_mcmc:",M_mcmc)
    print("d_mcmc:",d_mcmc)
    
    zd = np.percentile(samples[:, 0], [0.1,99.5])[0]
    zu = np.percentile(samples[:, 0], [0.1,99.5])[1]

    Td = np.percentile(samples[:, 1], [0.1,99.5])[0]
    Tu = np.percentile(samples[:, 1], [0.1,99.5])[1]

    Md = np.percentile(samples[:, 2], [0.1,99.5])[0]
    Mu = np.percentile(samples[:, 2], [0.1,99.5])[1]

    dd = np.percentile(samples[:, 3], [0.1,99.5])[0]
    du = np.percentile(samples[:, 3], [0.1,99.5])[1]

    fig = corner.corner(samples[:,0:4],
                    labels=["z", "T", r"$M$", r"$d$"],
                    quantiles=[0.16, 0.5, 0.84],range=[(zd,zu),(Td,Tu),(Md,Mu),(dd,du)],
                    show_titles=True,
                    truths=[z_fit,T_fit,M_fit,d_fit])
    
    plt.savefig("Model S 6 puntos {}.png".format(ID), dpi=300, format='png')
   
    return (z_mcmc,T_mcmc,M_mcmc,d_mcmc)

In [ ]:
z_mcmcT=[] #todos los redshift
T_mcmcT=[]
M_mcmcT=[]
d_mcmcT=[]
i_mcmcT=[]
ID_mcmcT=[]

In [ ]:
z_mcmcT6=[] #todos los redshift
T_mcmcT6=[]
M_mcmcT6=[]
d_mcmcT6=[]
i_mcmcT6=[]
ID_mcmcT6=[]

In [ ]:
for i in range(len(rows)):
    print(i)
    try:
        modeloS= mcmc_S(ID[i],i,frecsT_l[i],errfT_l[i],"S",30)
        z_mcmcT.append(modeloS[0])
        T_mcmcT.append(modeloS[1])
        M_mcmcT.append(modeloS[2])
        d_mcmcT.append(modeloS[3])
        i_mcmcT.append(i)
        ID_mcmcT.append(ID[i])
    except:
        pass
    
print(z_mcmcT)
print(T_mcmcT)
print(M_mcmcT)
print(d_mcmcT)
print(i_mcmcT)
print(ID_mcmcT)

In [ ]:
for i in range(len(rows)):
    print(i)
    try:
        modeloS6= mcmc_S6(ID[i],i,frecsT_l[i],errfT_l[i],"S",30)
        z_mcmcT6.append(modeloS6[0])
        T_mcmcT6.append(modeloS6[1])
        M_mcmcT6.append(modeloS6[2])
        d_mcmcT6.append(modeloS6[3])
        i_mcmcT6.append(i)
        ID_mcmcT6.append(ID[i])
    except:
        pass
    
print(z_mcmcT6)
print(T_mcmcT6)
print(M_mcmcT6)
print(d_mcmcT6)
print(i_mcmcT6)
print(ID_mcmcT6)

In [ ]:
zspec=[]

zmcmc=[]
ez1=[]
ez2=[]

Tmcmc=[]
eT1=[]
eT2=[]

Mmcmc=[]
eM1=[]
eM2=[]

dmcmc=[]
ed1=[]
ed2=[]

In [ ]:
zspec6=[]

zmcmc6=[]
ez1_6=[]
ez2_6=[]

Tmcmc6=[]
eT1_6=[]
eT2_6=[]

Mmcmc6=[]
eM1_6=[]
eM2_6=[]

dmcmc6=[]
ed1_6=[]
ed2_6=[]

In [ ]:
for item in i_mcmcT:
    zspec.append(z[item])

In [ ]:
for item in i_mcmcT6:
    zspec6.append(z[item])

In [ ]:
for (i,j,k,l) in zip(z_mcmcT,T_mcmcT,M_mcmcT,d_mcmcT):
    for (m,n,o,p) in zip(i,j,k,l):
        zmcmc.append(m[0])
        ez1.append(m[1])
        ez2.append(m[2])
        
        Tmcmc.append(n[0])
        eT1.append(n[1])
        eT2.append(n[2])
        
        Mmcmc.append(o[0])
        eM1.append(o[1])
        eM2.append(o[2])
        
        dmcmc.append(p[0])
        ed1.append(p[1])
        ed2.append(p[2])

In [ ]:
for (i,j,k,l) in zip(z_mcmcT6,T_mcmcT6,M_mcmcT6,d_mcmcT6):
    for (m,n,o,p) in zip(i,j,k,l):
        zmcmc6.append(m[0])
        ez1_6.append(m[1])
        ez2_6.append(m[2])
        
        Tmcmc6.append(n[0])
        eT1_6.append(n[1])
        eT2_6.append(n[2])
        
        Mmcmc6.append(o[0])
        eM1_6.append(o[1])
        eM2_6.append(o[2])
        
        dmcmc6.append(p[0])
        ed1_6.append(p[1])
        ed2_6.append(p[2])

In [ ]:
plt.hist(zspec,bins=10,edgecolor="k",alpha=0.7,label="$Z_{spec}$")
plt.hist(zmcmc,bins=10,edgecolor="k",alpha=0.7,label="$Z_{phot}$")
plt.title("Histogram redshifts (8 puntos)",fontsize=14)
plt.xlabel("redshift $z$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig("Histogram redshifts Model Single (8 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(zspec6,bins=10,edgecolor="k",alpha=0.7,label="$Z_{spec}$")
plt.hist(zmcmc6,bins=10,edgecolor="k",alpha=0.7,label="$Z_{phot}$")
plt.title("Histogram redshifts (6 puntos)",fontsize=14)
plt.xlabel("redshift $z$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig("Histogram redshifts Model Single (6 puntos).png", dpi=300, format='png')

In [ ]:
plt.figure(figsize=(12,9))

plt.scatter(zspec,zmcmc,c="k",marker="o")
plt.errorbar(zspec,zmcmc,[ez2,ez1],c="k",linestyle = 'None',linewidth=1)
plt.xlim([0,8])
plt.ylim([0,8])
plt.plot(zspec,zspec,c="r",ls="--")

plt.title("redshifts (spec vs phot) Model Single (8 puntos)",fontsize=14)
plt.xlabel(r"$z_{esp}$",fontsize=14)
plt.ylabel(r"$z_{phot}$",fontsize=14)
plt.savefig("redshifts (spec vs phot) Model Single (8 puntos).png", dpi=300, format='png')

In [ ]:
plt.figure(figsize=(12,9))

plt.scatter(zspec6,zmcmc6,c="b",marker="o")
plt.errorbar(zspec6,zmcmc6,[ez2_6,ez1_6],c="b",linestyle = 'None',linewidth=1)
plt.xlim([0,8])
plt.ylim([0,8])
plt.plot(zspec6,zspec6,c="r",ls="--")

plt.title("redshifts (spec vs phot) Model Single (6 puntos)",fontsize=14)
plt.xlabel(r"$z_{esp}$",fontsize=14)
plt.ylabel(r"$z_{phot}$",fontsize=14)
plt.savefig("redshifts (spec vs phot) Model Single (6 puntos).png", dpi=300, format='png')

In [ ]:
plt.figure(figsize=(12,9))

plt.scatter(zspec,zmcmc,c="k",marker="o",alpha=0.7,label="8 puntos")
plt.errorbar(zspec,zmcmc,[ez2,ez1],c="k",alpha=0.7,linestyle = 'None',linewidth=1)
plt.scatter(zspec6,zmcmc6,c="b",marker="o",alpha=0.7,label="6 puntos")
plt.errorbar(zspec6,zmcmc6,[ez2_6,ez1_6],c="b",alpha=0.7,linestyle = 'None',linewidth=1)
plt.xlim([0,8])
plt.ylim([0,8])
plt.plot(zspec,zspec,c="r",ls="--")

plt.title("redshifts (spec vs phot) Model Single (6 y 8 puntos)",fontsize=14)
plt.xlabel(r"$z_{esp}$",fontsize=14)
plt.ylabel(r"$z_{phot}$",fontsize=14)
plt.legend()
plt.savefig("redshifts (spec vs phot) Model Single (6 y 8 puntos).png", dpi=300, format='png')

In [ ]:
plt.figure(figsize=(12,9))

plt.scatter(zspec,np.array(zmcmc)-np.array(zspec),c="k",marker="o")
plt.errorbar(zspec,np.array(zmcmc)-np.array(zspec),[np.array(ez2)-np.array(zspec),np.array(ez1)-np.array(zspec)],c="k",linestyle = 'None',linewidth=1)
plt.axhline(0,c="r")

plt.title("Dif redshifts Model Single (8 puntos)",fontsize=14)
plt.xlabel(r"$z_{esp}$",fontsize=14)
plt.ylabel(r"$\Delta z$",fontsize=14)
plt.savefig("Dif redshifts Model Single (8 puntos).png", dpi=300, format='png')

In [ ]:
plt.figure(figsize=(12,9))

plt.scatter(zspec6,np.array(zmcmc6)-np.array(zspec6),c="b",marker="o")
plt.errorbar(zspec6,np.array(zmcmc6)-np.array(zspec6),[np.array(ez2_6)-np.array(zspec6),np.array(ez1_6)-np.array(zspec6)],c="b",linestyle = 'None',linewidth=1)
plt.axhline(0,c="r")

plt.title("Dif redshifts Model Single (6 puntos)",fontsize=14)
plt.xlabel(r"$z_{esp}$",fontsize=14)
plt.ylabel(r"$\Delta z$",fontsize=14)
plt.savefig("Dif redshifts Model Single (6 puntos).png", dpi=300, format='png')

In [ ]:
plt.figure(figsize=(12,9))

plt.scatter(zspec,np.array(zmcmc)-np.array(zspec),c="k",alpha=0.7,marker="o",label="8 puntos")
plt.errorbar(zspec,np.array(zmcmc)-np.array(zspec),[np.array(ez2)-np.array(zspec),np.array(ez1)-np.array(zspec)],c="k",alpha=0.7,linestyle = 'None',linewidth=1)
plt.scatter(zspec6,np.array(zmcmc6)-np.array(zspec6),c="b",alpha=0.7,marker="o",label="6 puntos")
plt.errorbar(zspec6,np.array(zmcmc6)-np.array(zspec6),[np.array(ez2_6)-np.array(zspec6),np.array(ez1_6)-np.array(zspec6)],c="b",alpha=0.7,linestyle = 'None',linewidth=1)
plt.axhline(0,c="r")

plt.title("Dif redshifts Model Single (6 y 8 puntos)",fontsize=14)
plt.xlabel(r"$z_{esp}$",fontsize=14)
plt.ylabel(r"$\Delta z$",fontsize=14)
plt.legend()
plt.savefig("Dif redshifts Model Single (6 y 8 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(np.array(zspec)-np.array(zmcmc),bins=10,edgecolor="k",label="$z$")
plt.title("Histogram dif redshifts Model Single (8 puntos)",fontsize=14)
plt.xlabel("diferencia $z$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
sigma8=np.std(np.array(zspec)-np.array(zmcmc))
plt.annotate("$\sigma$={}".format(np.round(sigma8,2)),(3,15),fontsize=14)
plt.savefig("Histogram dif redshifts Model Single (8 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(np.array(zspec6)-np.array(zmcmc6),bins=10,color="darkorange",edgecolor="k",label="$z$")
plt.title("Histogram dif redshifts Model Single (6 puntos)",fontsize=14)
plt.xlabel("diferencia $z$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
sigma6=np.std(np.array(zspec6)-np.array(zmcmc6))
plt.annotate("$\sigma$={}".format(np.round(sigma6,2)),(1.5,15),fontsize=14)
plt.savefig("Histogram dif redshifts Model Single (6 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(np.array(zspec)-np.array(zmcmc),bins=10,edgecolor="k",alpha=0.7,label="8 puntos, $\sigma$:{}".format(np.round(sigma8,2)))
plt.hist(np.array(zspec6)-np.array(zmcmc6),bins=10,edgecolor="k",alpha=0.7,label="6 puntos, $\sigma$:{}".format(np.round(sigma6,2)))
plt.title("Histogram dif redshifts Model Single (6 y 8 puntos)",alpha=0.7,fontsize=14)
plt.xlabel("diferencia $z$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig("Histogram dif redshifts Model Single (6 y 8 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(Tmcmc,bins=10,edgecolor="k",label="$T_{d}$")
plt.title("Histogram $T_{d}$ (8 puntos)",fontsize=14)
plt.xlabel("$T_{d}$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig("Histogram T Model Single (8 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(Tmcmc6,bins=10,color="darkorange",edgecolor="k",label="$T_{d}$")
plt.title("Histogram $T_{d}$ (6 puntos)",fontsize=14)
plt.xlabel("$T_{d}$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig("Histogram T Model Single (6 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(Tmcmc,bins=10,edgecolor="k",alpha=0.7,label="8 puntos")
plt.hist(Tmcmc6,bins=10,edgecolor="k",alpha=0.7,label="6 puntos")
plt.title("Histogram $T_{d}$ (6 puntos)",fontsize=14)
plt.xlabel("$T_{d}$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig("Histogram T Model Single (6 y 8 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(Mmcmc,bins=10,edgecolor="k",label="$M_{d}$")
plt.title("Histogram $M_{d}$ (8 puntos)",fontsize=14)
plt.xlabel("$M_{d}$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig("Histogram M Model Single (8 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(Mmcmc6,bins=10,color="darkorange",edgecolor="k",label="$M_{d}$")
plt.title("Histogram $M_{d}$ (6 puntos)",fontsize=14)
plt.xlabel("$M_{d}$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig("Histogram M Model Single (6 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(Mmcmc,bins=10,edgecolor="k",alpha=0.7,label="8 puntos")
plt.hist(Mmcmc6,bins=10,edgecolor="k",alpha=0.7,label="6 puntos")
plt.title("Histogram $M_{d}$ (6 y 8 puntos)",fontsize=14)
plt.xlabel("$M_{d}$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig("Histogram M Model Single (6 y 8 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(dmcmc,bins=10,edgecolor="k",label="$d$")
plt.title("Histogram $d$ (8 puntos)",fontsize=14)
plt.xlabel("$d$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig(r"Histogram d Model Single (8 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(dmcmc6,bins=10,color="darkorange",edgecolor="k",label="$d$")
plt.title("Histogram $d$ (6 puntos)",fontsize=14)
plt.xlabel("$d$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig(r"Histogram d Model Single (6 puntos).png", dpi=300, format='png')

In [ ]:
plt.hist(dmcmc,bins=10,edgecolor="k",alpha=0.7,label="8 puntos")
plt.hist(dmcmc6,bins=10,edgecolor="k",alpha=0.7,label="6 puntos")
plt.title("Histogram $d$ (6 y 8 puntos)",fontsize=14)
plt.xlabel("$d$",fontsize=14)
plt.ylabel("cantidad",fontsize=14)
plt.legend()
plt.savefig(r"Histogram d Model Single (6 y 8 puntos).png", dpi=300, format='png')